In [6]:
import os
import numpy as np
import pandas as pd
from scipy import interpolate
#coding=utf-8
#https://blog.csdn.net/iin729/article/details/71911240
import os
import linecache

In [8]:
XXdata=np.loadtxt('./data/坐标轴统一/45/45 (1).txt')
xxdata=np.loadtxt('./data/坐标轴统一/1/1 (1).txt')
xx=xxdata[:,0]
XX=XXdata[:,0]
yy=xxdata[:,1]
print(XX.shape)
print(xx.shape)
print(yy.shape)
s = interpolate.CubicSpline(xx, yy, bc_type='not-a-knot')
yyy = s(XX)
print(yyy.shape)
XXdata.shape

XX.shape
dirname = r'./data/坐标轴统一'
np.savetxt(dirname + "/" +"11" + r".txt", np.column_stack((XX, yyy)),
           fmt='%.4f\t%.4f')

(1340,)
(1340,)
(1340,)
(1340,)


In [282]:
root = r'./data/胃癌-3s/空白'
file_names = os.listdir(root)
file_ob_list = []
for file_name in file_names:  #循环地给这520个文件名加上它前面的路径，以得到它的具体路径
    fileob = root + '/' + file_name #文件夹路径加上\\ 再加上具体要读的的txt的文件名就定位到了这个txt
    file_ob_list.append(fileob)
print (file_ob_list)
ldata=[]
data = []
line_num = 1
total_line = len(open(file_ob_list[0]).readlines())
while line_num <= total_line:        #只有读完的行数小于等于总行数时才再读下一行，否则结束读取
    for file_ob in file_ob_list:    #按顺序循环读取所有文件
            line = linecache.getline(file_ob, line_num)#读取这个文件的第line_num行
            line = line.strip() #去掉这一行最后一个字符/n 即换行符
            if line is None or len(line) ==0 :
                break
            fields = line.split('\t')  #将这一行划分为两列，存放到列表中，fields是这样的： ['ENSG00000242268.2','0.0']
            prob = fields[1]
            # if fields[1] != 'NA':
            if fields[1].isdigit():  # 基因表达量不是NA也就是为数字时，才对它进行小数点的减少处理
                prob = float(fields[1])  # 将字符形式的数字如'0.0'强制转化为浮点型（带小数点的数字）数字0.0
                prob = '%.8f' % prob  # 只保留小数点后面的4位小数

            if file_ob == file_ob_list[0]:  # 如果读的是第一个txt文件，则将读进去的第一列基因名和第二列表达量
                data = [fields[0], prob]  # 都加入到列表中  data=  ['ENSG00000242268.2','0.0']
            else:  # 如果读进去的不是第一个文件，则跳到else执行，第一列不要，
                data.append(prob)

    line_num = line_num + 1  # 行数加1，好接着读取每一个文件的第二行
    ldata.append(
        data)  # 将存放了所有txt的第一行数据的data，放到一个新的列表中保存，这时ldata就是一个二维列表，ldata=[['ENSG00000242268.2','0.0'，'0.10'],[...],...]
    # 用来存放所有的（60482行，就是所有的基因名）行数
    data = []
f = open( r'./data/坐标轴统一/mapping-胃癌/gastric cancer-00000000.txt', "w+")  # 创建存放数据的文件
for i, p in enumerate(
        ldata):  # 将数据写入文件,i是enumerate()函数返回的ldata的某个元素p(就是一行数据，如['ENSG00000242268.2','0.0'，'0.10']从第一个开始)开始的序号（0,1,2等）
    for j, q in enumerate(p):  # 读取p（如['ENSG00000242268.2','0.0'，'0.10']）中的每一个元素
        f.write(q + "\t")  # 将这个元素写到txt中，每写一个加入一个“\t”（它代表excel中的一根竖线）
    print
    i  # 显示一下打印到了第多少行
    f.write("\n")  # 每写完一行，就写入一个换行符"\n"，好使的接下来的数据写入到第二行


f.close()  # 操作完一个文件后应该将它关闭


['./data/胃癌-3s/空白/00000000 (1).txt', './data/胃癌-3s/空白/00000000 (10).txt', './data/胃癌-3s/空白/00000000 (2).txt', './data/胃癌-3s/空白/00000000 (3).txt', './data/胃癌-3s/空白/00000000 (4).txt', './data/胃癌-3s/空白/00000000 (5).txt', './data/胃癌-3s/空白/00000000 (6).txt', './data/胃癌-3s/空白/00000000 (7).txt', './data/胃癌-3s/空白/00000000 (8).txt', './data/胃癌-3s/空白/00000000 (9).txt']


In [283]:


def splitMapping(dirname,flg):
    '''
    实现将文件夹下所有mapping数据拆分成单谱
    '''
    for category in os.listdir(dirname):
        # 文件夹名
        if (category[-3:] == 'txt'):
            # 判断文件是不是
            os.mkdir(dirname+"/"+category[:-4])
            # 创建新目录
            catdir = os.path.join(dirname,category)
            # print(catdir)
            if flg == 0:
                # 不删除首行
                data = np.array(pd.read_csv(catdir,delim_whitespace=True, header=None))
                x = data[:,0]
                print(x.shape)
                for index in range(1, data.shape[1]):
                    y = data[:, index]
                    s = interpolate.CubicSpline(x, y, bc_type='not-a-knot')
                    yy = s(XX)
                    print(yy.shape)
                    np.savetxt(dirname + "/" + category[:-4] +"/"+ str(index).rjust(4, '0')+ "_" +
                               category[:-4] + r".txt", np.column_stack((XX, yy)),
                               fmt='%.4f\t%.4f')
            else:
                # 删除首行
                data = np.array(pd.read_csv(catdir, delim_whitespace=True, header=1))
                x = data[:, 0]
                for index in range(1, data.shape[1]):
                    np.savetxt(dirname + "/" + category[:-4] + "/" + str(index).rjust(4, '0') + "_" +
                               category[:-4] + r".txt",np.column_stack((x, data[:, index])),
                               fmt='%i\t%.4f')
                    # np.savetxt(dirname + "\\" + category[:-4] + "\\" + str(index).rjust(4, '0') + "_" +
                    #            category[:-4] + r".txt", np.column_stack((x, data[:, index])),
                    #            fmt='%.2f\t%i')
        else:
            continue

    return 1


dirname = r'./data/坐标轴统一/mapping-胃癌'
#文件夹下所有mapping数据拆分

flg = 0
#flg = 0：不删除首行
splitMapping(dirname,flg)

(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)
(1340,)


1